# Task 1: Minimal media / essential uptakes
We want to investigate minimal media / essential uptake reactions
* A minimal medium is a medium (i.e. a set of uptake reactions with lower
bounds < 0) that enables growth (otherwise it’s not a medium). Setting any
single one of these uptakes to zero will result in zero growth (otherwise it’s
not minimal)
* An essential uptake reaction (= essential component in the medium) is a
reaction that will lead to zero growth if removed, regardless of what else
can be taken up

There are several approaches to achieve this, three are outlined below:

**Strategy A:**
* Start with all uptakes enabled, switch off one uptake and check if growth is
still possible
* Switch the uptake back on and test the next component

**Strategy B:**
* Enable all uptakes, switch one off and check if growth is possible.
* If growth is still possible, remove the next one, otherwise switch it back on
and then remove the next one
* Repeat until all uptakes were tested

**Strategy C:**
* Set up a linear program to minimize the number of uptake reactions that enables growth
* Technical detail: This is not the same as minimizing the sum of fluxes of exchange
reactions. Minimizing the number of active exchange reactions requires a Mixed Integer
Linear Program (MILP)
* This approach is implemented in COBRApy: cobra.medium.minimal_medium(model,
growth_rate, minimize_components=True)
    + The variable growth_rate does not influence the result as long as the value is “reasonable”, such
as, for example, 1
    + Reasonable means that it can’t be so small as to cause numerical problems and not so large that
fluxes would be limited by the default bounds

# Some general points mentioned by Lecturer

There are different flavours of reactions that exchange components between the cell
and its surroundings (steady-state) and a reservoir (not steady-state)
* As mentioned in the lecture, a lot of these have IDs that start with EX_ and are called exchange
reactions, but there are also other types – we will ignore the latter
* For the exercises, only consider reactions whose ID starts with EX_
    + You can iterate over all reactions and check if the ID starts with EX_ or find COBRApy’s built-in function to
achieve this
    + Keep in mind that there can be numerical issues, consider fluxes below 10-5 to be zero
    + If you are only interested in the value of the objective function, i.e. you don’t need the
other fluxes, you can use slim_optimize() instead of optimize(), which may speed up the
calculations a bit

# Initialization

Loading package and model and have a first look into it.

Further:

1) Set the lower bound for all exchange
reactions (IDs starting with EX_) to -1000
2) Make sure that the biomass reactions
( “BIOMASS_Ec_iAF1260_core_59p81M” for
iAF1260 and “BIOMASS_Ecoli_core_w_GAM”
for coli_core) are set as the objective functions,
respectively

In [ ]:
# ---------------------------------------------
# If necessary, this installs cobra using pip
!pip install cobra
# Documentation ------------------------------
# https://cobrapy.readthedocs.io/en/latest/

import cobra

In [ ]:
### reading models ---------------------------
# Core Model of E. coli
# Organism: Escherichia coli str. K-12 substr. MG1655
# Download here: http://bigg.ucsd.edu/models/e_coli_core
# C = Core
model_c = cobra.io.read_sbml_model("task1_e_coli_core.xml")

# Genome Scale Model of E. coli
# Organism: Escherichia coli str. K-12 substr. MG1655
# Download here: http://bigg.ucsd.edu/models/iAF1260
# GS = Genomice Scale
model_gs = cobra.io.read_sbml_model("task1_iAF1260.xml")

In [ ]:
# brief overview of core model
model_c

Name,e_coli_core
Memory address,7d7bdedde9e0
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [ ]:
# brief overview of genome scale model
model_gs

Name,iAF1260
Memory address,7be5468e7010
Number of metabolites,1668
Number of reactions,2382
Number of genes,1261
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iAF1260_core_59p81M - 1.0*BIOMASS_Ec_iAF1260_core_59p81M_reverse_3925e
Compartments,"cytosol, periplasm, extracellular space"


In [ ]:
# defining objectives
model_c.objective = "BIOMASS_Ecoli_core_w_GAM"
model_gs.objective = "BIOMASS_Ec_iAF1260_core_59p81M"

In [ ]:
# finding and extracting exchange reactions
model_c_exchange_reactions = [rxn for rxn in model_c.reactions if rxn.id.startswith("EX_")]
model_gs_exchange_reactions = [rxn for rxn in model_gs.reactions if rxn.id.startswith("EX_")]

# Set the lower bound for all exchange reactions (IDs starting with EX_) to -1000
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

Here we are searching for all the

# Task 1.1 (7 points)
Implement strategy A:
* Start with all uptakes enabled, switch off one uptake and check if growth is
still possible
* Switch the uptake back on and test the next component

**Questions to be answered:**
* Does enabling all uptakes identified as essential (and
switching off the uptake for all other exchange reactions)
lead to a valid growth medium?
* Does this approach guarantee that you find a valid growth
medium? Why (not)?
* Is the result necessarily unique? If not, what could it
depend on?

In [ ]:
# This Block tests if there is growth possible or not
def identify_essential_exchange_reactions_task1(model, exchange_reactions):
    essential_uptakes = []
    initial_growth = model.slim_optimize()

    print(f"Initial growth: {initial_growth}")

    for rxn in exchange_reactions:
        original_bound = rxn.lower_bound
        rxn.lower_bound = 0
        growth = model.slim_optimize()
        #print(f"Reaction: {rxn.id}; Growth: {growth}")
        if growth < 1e-5:
           essential_uptakes.append(rxn.id)
           print(f"YES - Reaction {rxn.id} is at least probably essential due to no given growth; growth value: {growth}")
        else:
          print(f"NO - Reaction {rxn.id} is at least probably NOT essential; growth value: {growth}")
        rxn.lower_bound = original_bound
    return essential_uptakes

In [ ]:
#---------------------------------------------------------------------
# As I started several tests and recognised that after the first test everything
# fails if I do not execute the commands again in between "----" I decided
# to include those, even if not necessary at the first point, just to be safe

# finding and extracting exchange reactions
model_c_exchange_reactions = [rxn for rxn in model_c.reactions if rxn.id.startswith("EX_")]
model_gs_exchange_reactions = [rxn for rxn in model_gs.reactions if rxn.id.startswith("EX_")]

# Set the lower bound for all exchange reactions (IDs starting with EX_) to -1000
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake
#---------------------------------------------------------------------

print("\n====================================================================")
print("Identification of essential uptakes in the core model!")
print("====================================================================\n")
essential_uptakes_c_t1 = identify_essential_exchange_reactions_task1(model_c, model_c_exchange_reactions)

# Deactivation of all exchange reactions
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = 0

# Activation of only those exchange reactions being indicated by essential_uptakes
for rxn_id  in essential_uptakes_c_t1:
     model_c.reactions.get_by_id(rxn_id).lower_bound = -1000

print("\n====================================================================")
print("Overview of the uptakes assumed to be essential in the core model\n:")
print(essential_uptakes_c_t1)

print("\n====================================================================")
print("Overview of the set bounds depending on the assumed essential uptakes:\n")
for rxn in model_c_exchange_reactions:
    print(f"Reaction: {rxn.id}; Lower Bound: {rxn.lower_bound}")

growth_rate_c_t1 = model_c.slim_optimize()

# Check if growth is possible
print("\n====================================================================")
print(f"Calculated growth using only the uptakes {essential_uptakes_c_t1}\nGrowth: {growth_rate_c_t1}")

if growth_rate_c_t1 >= 1e-5:
    print("\nEnabling only the essential uptakes leads to a valid growth medium.")
    print(f"Growth rate: {growth_rate_c_t1}")
else:
    print("\nEnabling only the assumed essential uptakes does NOT lead to a \nvalid growth medium.")
    print("Growth is not possible with only the essential uptakes.")

In [ ]:
#---------------------------------------------------------------------
# As I started several tests and recognised that after the first test everything
# fails if I do not execute the commands again in between "----" I decided
# to include those, even if not necessary at the first point, just to be safe

# finding and extracting exchange reactions
model_c_exchange_reactions = [rxn for rxn in model_c.reactions if rxn.id.startswith("EX_")]
model_gs_exchange_reactions = [rxn for rxn in model_gs.reactions if rxn.id.startswith("EX_")]

# Set the lower bound for all exchange reactions (IDs starting with EX_) to -1000
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

#---------------------------------------------------------------------

print("\n====================================================================")
print("Identification of essential uptakes in the genomic scale model!")
print("====================================================================\n")
essential_uptakes_gs_t1 = identify_essential_exchange_reactions_task1(model_gs, model_gs_exchange_reactions)

# Deactivation of all exchange reactions
for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = 0

# Activation of only those exchange reactions being indicated by essential_uptakes
for rxn_id  in essential_uptakes_gs_t1:
     model_gs.reactions.get_by_id(rxn_id).lower_bound = -1000

print("\n====================================================================")
print("Short overview of the uptakes assumed to be essential in the core model\n:")
print(essential_uptakes_gs_t1)

print("\n====================================================================")
print("Overview of the set bounds depending on the assumed essential uptakes:\n")
for rxn in model_gs_exchange_reactions:
    print(f"Reaction: {rxn.id}; Lower Bound: {rxn.lower_bound}")

growth_rate_gs_t1 = model_gs.slim_optimize()

# Check if growth is possible
print("\n====================================================================")
print(f"Calculated growth using only the uptakes {essential_uptakes_gs_t1}; Growth: {growth_rate_gs_t1}")

if growth_rate_gs_t1 >= 1e-5:
    print("\nEnabling only the essential uptakes leads to a valid growth medium.")
    print(f"Growth rate: {growth_rate_gs_t1}")
else:
    print("\nEnabling only the assumed essential uptakes does NOT lead to a valid growth medium.")
    print("Growth is not possible with only the essential uptakes.")

## Answers of Task 1.1:
**Does enabling all uptakes identified as essential (and
switching off the uptake for all other exchange reactions)
lead to a valid growth medium?**

At least for the two models tested enabling all the as essential identified uptakes and switching off the others did not lead to a valid growth medium. The growth rate could not even be calculated.

**Does this approach guarantee that you find a valid growth
medium? Why (not)?**

As we could see in our tests above it did not guarantee a valid growth medium. In the lecture we have learnt that there might be pathways which need more than one nutrient to work sufficiently. Further, when each nutrient is individually removed, some may appear essential as their absence alone halts the growth. However, as in the case above, when multiple non-essential nutrients are simultaneosly absent, the combined effect may still prevent growth due to metabolic dependencies (as written above) or the need of alternative pathways which may need these nutrients.

**Is the result necessarily unique? If not, what could it
depend on?**

The outcome highly depends on the model and the network inside used. Different models or even different versions of the same model might have variations in reaction definitions, reversibility, and constraints, leading to different sets of essential nutrients. Additionally, computational factors such as solver tolerances and numerical precision can influence the results. Since metabolic networks often have multiple pathways to achieve growth, there may be several valid combinations of nutrients that support growth.

# Task 1.2 (7 points)
**Implement Strategy B:**
* Enable all uptakes, switch one off and check if growth is possible.
* If growth is still possible, remove the next one, otherwise switch it back on
and then remove the next one
* Repeat until all uptakes were tested

**Questions to be answered:**
* Does it guarantee a valid growth medium?
* Does it guarantee a medium with a minimal number
of components?
* Is the result necessarily unique? If not, what could it
depend on?

In [ ]:
#---------------------------------------------------------------------
# As I started several tests and recognised that after the first test everything
# fails if I do not execute the commands again in between "----" I decided
# to include those, even if not necessary at the first point, just to be safe

# finding and extracting exchange reactions
model_c_exchange_reactions = [rxn for rxn in model_c.reactions if rxn.id.startswith("EX_")]
model_gs_exchange_reactions = [rxn for rxn in model_gs.reactions if rxn.id.startswith("EX_")]

# Set the lower bound for all exchange reactions (IDs starting with EX_) to -1000
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake
#---------------------------------------------------------------------

In [ ]:
# This Block tests if there is growth possible or not
def identify_essential_exchange_reactions_task2(model, exchange_reactions):
    essential_uptakes = []
    initial_growth = model.slim_optimize()

    print(f"Initial growth: {initial_growth}")

    for rxn in exchange_reactions:
        original_bound = rxn.lower_bound
        rxn.lower_bound = 0
        growth = model.slim_optimize()
        if growth < 1e-5:
           essential_uptakes.append(rxn.id)
           rxn.lower_bound = original_bound
           print(f"YES - Reaction {rxn.id} is at least probably essential due to no given growth; growth value: {growth}")
           print(f"The Reaction {rxn.id} is kept (bound reset to original value)")
        else:
          print(f"NO - Reaction {rxn.id} is at least probably NOT essential; growth value: {growth}")
          print("The bound = 0 is not changed")

    return essential_uptakes

In [ ]:
#---------------------------------------------------------------------
# As I started several tests and recognised that after the first test everything
# fails if I do not execute the commands again in between "----" I decided
# to include those, even if not necessary at the first point, just to be safe

# finding and extracting exchange reactions
model_c_exchange_reactions = [rxn for rxn in model_c.reactions if rxn.id.startswith("EX_")]
model_gs_exchange_reactions = [rxn for rxn in model_gs.reactions if rxn.id.startswith("EX_")]

# Set the lower bound for all exchange reactions (IDs starting with EX_) to -1000
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake
#---------------------------------------------------------------------

print("\n====================================================================")
print("Identification of essential uptakes in the core model!")
print("====================================================================\n")
essential_uptakes_c_t2 = identify_essential_exchange_reactions_task2(model_c, model_c_exchange_reactions)

# Deactivation of all exchange reactions
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = 0

# Activation of only those exchange reactions being indicated by essential_uptakes
for rxn_id  in essential_uptakes_c_t2:
     model_c.reactions.get_by_id(rxn_id).lower_bound = -1000

print("\n====================================================================")
print("Overview of the uptakes assumed to be essential in the core model\n:")
print(essential_uptakes_c_t2)

print("\n====================================================================")
print("Overview of the set bounds depending on the assumed essential uptakes:\n")
for rxn in model_c_exchange_reactions:
    print(f"Reaction: {rxn.id}; Lower Bound: {rxn.lower_bound}")

growth_rate_c_t2 = model_c.slim_optimize()

# Check if growth is possible
print("\n====================================================================")
print(f"Calculated growth using only the uptakes {essential_uptakes_c_t2}\nGrowth: {growth_rate_c_t2}")

if growth_rate_c_t2 >= 1e-5:
    print("\nEnabling only the essential uptakes leads to a valid growth medium.")
    print(f"Growth rate: {growth_rate_c_t2}")
else:
    print("\nEnabling only the assumed essential uptakes does NOT lead to a valid growth medium.")
    print("Growth is not possible with only the essential uptakes.")

In [ ]:
#---------------------------------------------------------------------
# As I started several tests and recognised that after the first test everything
# fails if I do not execute the commands again in between "----" I decided
# to include those, even if not necessary at the first point, just to be safe

# finding and extracting exchange reactions
model_c_exchange_reactions = [rxn for rxn in model_c.reactions if rxn.id.startswith("EX_")]
model_gs_exchange_reactions = [rxn for rxn in model_gs.reactions if rxn.id.startswith("EX_")]

# Set the lower bound for all exchange reactions (IDs starting with EX_) to -1000
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake
#---------------------------------------------------------------------

print("\n====================================================================")
print("Identification of essential uptakes in the core model!")
print("====================================================================\n")
essential_uptakes_gs_t2 = identify_essential_exchange_reactions_task2(model_gs, model_gs_exchange_reactions)

# Deactivation of all exchange reactions
for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = 0

# Activation of only those exchange reactions being indicated by essential_uptakes
for rxn_id  in essential_uptakes_gs_t2:
     model_gs.reactions.get_by_id(rxn_id).lower_bound = -1000

print("\n====================================================================")
print("Overview of the uptakes assumed to be essential in the core model\n:")
print(essential_uptakes_gs_t2)

print("\n====================================================================")
print("Overview of the set bounds depending on the assumed essential uptakes:\n")
for rxn in model_gs_exchange_reactions:
    print(f"Reaction: {rxn.id}; Lower Bound: {rxn.lower_bound}")

growth_rate_gs_t2 = model_gs.slim_optimize()

# Check if growth is possible
print("\n====================================================================")
print(f"Calculated growth using only the uptakes {essential_uptakes_gs_t2}\nGrowth: {growth_rate_gs_t2}")

if growth_rate_gs_t2 >= 1e-5:
    print("\nEnabling only the essential uptakes leads to a valid growth medium.")
    print(f"Growth rate: {growth_rate_gs_t2}")
else:
    print("\nEnabling only the assumed essential uptakes does NOT lead to a valid growth medium.")
    print("Growth is not possible with only the essential uptakes.")

## Answers of Task 1.2:
**Does it guarantee a valid growth medium?**

Obviously also this approach does not lead to a valid growth medium, even if the result was positive on the genomic scale model, it was not on the core model. For Task 1 the endpoints were even different EX_pi_e vs. EX_nh4_e, while for genomic scale model there were found more essential uptakes:

```
task 1.1
['EX_ca2_e', 'EX_cl_e', 'EX_cobalt2_e', 'EX_mg2_e', 'EX_mn2_e', 'EX_mobd_e', 'EX_zn2_e', 'EX_k_e']

task 1.2
['EX_ca2_e', 'EX_cl_e', 'EX_cobalt2_e', 'EX_cu_e', 'EX_mg2_e', 'EX_mn2_e',
'EX_mobd_e', 'EX_o2_e', 'EX_feoxam_e', 'EX_zn2_e', 'EX_imp_e', 'EX_isetac_e', 'EX_k_e']
```


**Does it guarantee a medium with a minimal number of components?**

If the result is not positive in both cases, there cannot be any guarantee. Further, there is a real great bias in that method: The uptakes are reset sequentially, which means there is a order. If, just by chance, the "wrong" reaction is deactivated this leads to wrong assumptions. Additionally all the pathways which rely on two or even more uptakes might only work if all of those are active. Probably it would be useful to run this multiple times (e.g. permutational based) with different orders of the uptakes and to randomly deactive several reactions to see what components or reactions are really necessary when having a great overlook about all the tests.

**Is the result necessarily unique? If not, what could it depend on?**

No it is not necessarily unique. It depends on the complexity of the model (the more complex a model is the more alternative pathways might be possible). Further, if the order of the reactions would be different the outcome might differ.


# Task 1.3 (6 Points)
**Use the built-in method for Strategy C:**
* Set up a linear program to minimize the number of uptake reactions that enables growth
* Technical detail: This is not the same as minimizing the sum of fluxes of exchange
reactions. Minimizing the number of active exchange reactions requires a Mixed Integer
Linear Program (MILP)
* This approach is implemented in COBRApy: cobra.medium.minimal_medium(model,
growth_rate, minimize_components=True)
    + The variable growth_rate does not influence the result as long as the value is “reasonable”, such
as, for example, 1
    + Reasonable means that it can’t be so small as to cause numerical problems and not so large that
fluxes would be limited by the default bounds

In [ ]:
# ----------------------------------------------------------------------
# finding and extracting exchange reactions
model_c_exchange_reactions = [rxn for rxn in model_c.reactions if rxn.id.startswith("EX_")]
model_gs_exchange_reactions = [rxn for rxn in model_gs.reactions if rxn.id.startswith("EX_")]

# Set the lower bound for all exchange reactions (IDs starting with EX_) to -1000
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake
#---------------------------------------------------------------------------

# Core Model ----------

growth_rate = 1.0

model_c_max_growth = model_c.slim_optimize()

minimal_media = cobra.medium.minimal_medium(model_c, growth_rate, minimize_components=True)
print(minimal_media)


# Set all exchange reactions to zero uptake
for rxn in model_c.exchanges:
    rxn.lower_bound = 0.0

# Set the uptake fluxes for the minimal medium components
for metabolite_id, uptake_value in minimal_media.items():
    exchange_rxn = model_c.exchanges.get_by_id(f"{metabolite_id}")
    exchange_rxn.lower_bound = -uptake_value  # Negative because uptake is negative flux

# Verify that the model can achieve the specified growth rate
solution = model_c.optimize()

print(f"\nGrowth rate with minimal medium: {solution.objective_value}")

EX_glu__L_e    332.395406
EX_o2_e        500.000000
EX_pi_e         55.795559
dtype: float64

Growth rate with minimal medium: 15.167194694546339


In [ ]:
# ----------------------------------------------------------------------
# finding and extracting exchange reactions
model_c_exchange_reactions = [rxn for rxn in model_c.reactions if rxn.id.startswith("EX_")]
model_gs_exchange_reactions = [rxn for rxn in model_gs.reactions if rxn.id.startswith("EX_")]

# Set the lower bound for all exchange reactions (IDs starting with EX_) to -1000
for rxn in model_c_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake

for rxn in model_gs_exchange_reactions:
    rxn.lower_bound = -1000  # Allow uptake
#---------------------------------------------------------------------------

# Core Model ----------

growth_rate = 1.0

model_gs_max_growth = model_gs.slim_optimize()

minimal_media = cobra.medium.minimal_medium(model_gs, growth_rate, minimize_components=True)
print(minimal_media)


# Set all exchange reactions to zero uptake
for rxn in model_gs.exchanges:
    rxn.lower_bound = 0.0

# Set the uptake fluxes for the minimal medium components
for metabolite_id, uptake_value in minimal_media.items():
    exchange_rxn = model_gs.exchanges.get_by_id(f"{metabolite_id}")
    exchange_rxn.lower_bound = -uptake_value  # Negative because uptake is negative flux

# Verify that the model can achieve the specified growth rate
solution = model_gs.optimize()

print(f"\nGrowth rate with minimal medium: {solution.objective_value}")

EX_amp_e        1000.000000
EX_ca2_e           0.004737
EX_cl_e            0.004737
EX_cobalt2_e       0.003158
EX_cu2_e           0.003158
EX_mg2_e           0.007895
EX_mn2_e           0.003158
EX_mobd_e          0.003158
EX_fe3dcit_e    1000.000000
EX_zn2_e           0.003158
EX_so4_e           0.250250
EX_k_e             0.177600
dtype: float64

Growth rate with minimal medium: 1.0000000000000004


## Answers of Task 1.3:
**Does it result in the same components as strategies A and B?**

Yes, the result differs greatly from both other strategies A and B.
Using strategie A it was not possible to "create" a medium in which the organism could have grown.
Using strategy B at least in the genomic scale model it was possible to get a working medium, but not with the values of the core model.
In this strategy, namely C, we got for both models components which led to a minimum medium which at least most likely will work - assuming that the reactions we have filtered out for the caluclation are enough for such calculations.

Further, comparing the core model, we got three components using strategy C, while in strategy A and B we always got only one. What is interesting is, that the component EX_nh4_e did not appear in strategy C but only in strategy B (task 1.2).

When comparing the results for the genomic scale model, there were some uptakes which were identified in all strategies, while others seemed to be found just by accident - please see table below:

| **Strategy A**   | **Strategy B**   | **Strategy C**   | **always found?** |
|------------------|------------------|------------------|-------------------|
| EX_ca2_e         | EX_ca2_e         | x                | -                 |
| x                | x                | EX_amp_e         | -                 |
| x                | x                | EX_ca2_e         | -                 |
| **EX_cl_e**      | **EX_cl_e**      | **EX_cl_e**      | **yes**           |
| **EX_cobalt2_e** | **EX_cobalt2_e** | **EX_cobalt2_e** | **yes**           |
| x                | EX_cu_e          | EX_cu2_e         | -                 |
| **EX_k_e**       | **EX_k_e**       | **EX_k_e**       | **yes**           |
| x                | EX_feoxam_e      |                  | -                 |
| x                | x                | EX_fe3dcit_e     | -                 |
| **EX_mg2_e**     | **EX_mg2_e**     | **EX_mg2_e**     | **yes**           |
| x                | EX_imp_e         | x                | -                 |
| **EX_mn2_e**     | **EX_mn2_e**     | **EX_mn2_e**     | **yes**           |
| **EX_mobd_e**    | **EX_mobd_e**    | **EX_mobd_e**    | **yes**           |
| **EX_zn2_e**     | **EX_zn2_e**     | **EX_zn2_e**     | **yes**           |
| x                | EX_isetac_e      | x                | -                 |
| x                | EX_o2_e          | x                | -                 |
| x                | x                | EX_so4_e         | -                 |


**If yes, is this necessarily the case? If no, why not?**

No, as for strategy A and B, it was just a testing of components without any system behind which is more complex than: if it works, put it on again, else, throw it away (at least in mind). That is not the case for strategy C which is much more complex.

**Is the result of strategy C necessarily unique? If no, what
could it depend on?**

Not it is not, as there might be many different alternative pathways and thus solutions which might end up with the objective value. Thus, different combinations of nutrients might equally satisfy the growth requirements while the number of components is minimized. This means there could be several minimal media with the same minimal number of uptake reactions but with different nutrient compositions.

**Which of the three strategies would you choose to find a
minimal medium?**

As only strategy C worked well, I would choose strategy C. Further, it works systematically and not like "brute force" in strategys A and B.